In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/StackedDataAugmentation

## turning dataloader into .py and moving it to GANTraining folder
# !jupyter nbconvert --to python "2.EEGMMIDBDataloader.ipynb"

# move 2.EEGMMIDBDataloader.py to GANTraining folder
#%mv EEGMMIDBDataloader.py GANTraining/
# %cd GANTraining/
# !ls

In [11]:
# imports
from EEGMMIDBDatasetLoaderV2 import EEGMMIDBDataset
from torch.utils.data import DataLoader
from GANModels import Generator, Discriminator
from functions import train, copy_params, load_params, LinearLrDecay
from utils.utils import save_checkpoint
from torch.utils.tensorboard import SummaryWriter

import torch
import numpy as np
import os
import io
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import PIL.Image
from copy import deepcopy

/Users/ryan/Desktop/StackedDataAugmentation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [18]:
import sys
import train_GAN
import os
from tqdm import tqdm
from glob import glob
import re


# Base configuration (edit as needed)
base_data_path = '/Users/ryan/Desktop/StackedDataAugmentation/physionetdata/eegmmidb_train_df.pkl'
checkpoint_root = "GANTraining/logs"

# List of all classes to train
# eeg_classes = ['left_hand', 'right_hand', 'both_hands', 'both_feet', 'rest']
eeg_classes = ['left_hand']


num_epochs_per_batch = 50

# Shared hyperparameters
shared_args = {
    '--batch_size': '32',
    '--max_epoch': '50',
    '--max_iter': '18000',
    '--latent_dim': '100',
    '--channels': '64',
    '--img_size': '640',
    '--g_lr': '0.00028',
    '--d_lr': '0.00023',
    '--n_critic': '5',
    '--exp_name': '',  # placeholder, will be set per class
    '--class_name': '',  # placeholder, will be set per class
    '--data_path': base_data_path,
    '--grow_steps': '0',
}



for cls in tqdm(eeg_classes, desc='Training GAN per class'):
    print(f"\n=== 🔁 Starting class: {cls} ===")

    # Reset possible load_path from prior loop
    shared_args.pop('--load_path', None)

    clean_cls = cls.replace(" ", "_")
    save_dir = os.path.join(checkpoint_root, f"{clean_cls}_GAN_checkpoint")
    os.makedirs(save_dir, exist_ok=True)

    shared_args['--class_name'] = cls
    shared_args['--exp_name'] = f"GAN_{clean_cls}_exp"

    # Build sys.argv
    sys.argv = ['train_GAN.py']
    for k, v in shared_args.items():
        sys.argv.extend([k, v])

    # Train
    train_GAN.main()

Training GAN per class:   0%|          | 0/1 [00:00<?, ?it/s]


=== 🔁 Starting class: left_hand ===
using CPU, this will be slow
Discriminator(
  (0): PatchEmbedding_Linear(
    (projection): Sequential(
      (0): Rearrange('b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=1, s2=20)
      (1): Linear(in_features=1280, out_features=1280, bias=True)
    )
  )
  (1): Dis_TransformerEncoder(
    (0): Dis_TransformerEncoderBlock(
      (0): ResidualAdd(
        (fn): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (keys): Linear(in_features=1280, out_features=1280, bias=True)
            (queries): Linear(in_features=1280, out_features=1280, bias=True)
            (values): Linear(in_features=1280, out_features=1280, bias=True)
            (att_drop): Dropout(p=0.5, inplace=False)
            (projection): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (2): Dropout(p=0.5, inplace=False)
        )
      )
      (1): ResidualAdd(
        (fn): Seque

Training GAN per class:   0%|          | 0/1 [00:07<?, ?it/s]


TypeError: type torch.cuda.FloatTensor not available. Torch not compiled with CUDA enabled.